In [ ]:
# @title Environment setup and simulator import
import os
import sys
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Define the Drive and Local paths.
DRIVE_FOLDER = "/content/drive/My Drive/projects/TensorMorph"
LOCAL_FOLDER = "/content/tensormorph_local"

# Mount Google Drive to access the source files.
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

# Synchronize Drive to the local Colab environment for faster access.
if not os.path.exists(LOCAL_FOLDER):
    os.makedirs(LOCAL_FOLDER)

# Use rsync to copy the experimental folder locally.
!rsync -av --progress "{DRIVE_FOLDER}/experimental/" "{LOCAL_FOLDER}/experimental/"

# Move into the local directory and update the path.
os.chdir(LOCAL_FOLDER)
sys.path.append(LOCAL_FOLDER)
sys.path.append(os.path.join(LOCAL_FOLDER, "experimental"))

try:
    from simulator import HardwareScenario
    print("Simulator successfully loaded from local copy.")
except ImportError as e:
    print(f"Error: could not find simulator.py in {LOCAL_FOLDER}/experimental. {e}")

# Create the data directory for scenario storage.
os.makedirs("data", exist_ok=True)

print(f"Working directory set to: {os.getcwd()}")
print("Environment ready.")

In [ ]:
# @title Feature vector generation

def generate_random_scenarios(n=10000):
    # Dimensions common in standard vision models.
    spatial_dims = [7, 14, 28, 56, 112, 224]
    channels = [3, 16, 24, 32, 48, 64, 96, 128, 256, 512, 1024]

    data = []
    for _ in range(n):
        dim = np.random.choice(spatial_dims)
        ic = np.random.choice(channels)

        # Probabilistic split between standard and depthwise.
        is_dw = np.random.choice([0, 1], p=[0.7, 0.3])
        oc = ic if is_dw else np.random.choice(channels)

        k = np.random.choice([1, 3, 5, 7])
        s = np.random.choice([1, 2])

        # Length of the elementwise operation chain.
        chain_len = np.random.randint(1, 6)

        # Indicator for a trailing activation op.
        has_act = np.random.choice([0, 1])

        data.append({
            'in_h': dim, 'in_w': dim, 'in_c': ic, 'out_c': oc,
            'kernel': k, 'stride': s, 'is_dw': is_dw,
            'chain_len': chain_len, 'has_act': has_act
        })

    return pd.DataFrame(data)

# Generate the initial pool of scenarios.
raw_df = generate_random_scenarios(15000)
print(f"Pool size: {len(raw_df)}")
raw_df.head()

In [ ]:
# @title Scenario generation and labeling

# Memory-Bound scenario: low bandwidth relative to compute.
mem_scenario = HardwareScenario(
    "Memory_Bound",
    flops_per_sec=1e12,
    bytes_per_sec=1e9
)

# Compute-Bound scenario: low compute relative to bandwidth.
compute_scenario = HardwareScenario(
    "Compute_Bound",
    flops_per_sec=1e9,
    bytes_per_sec=1e12
)

def label_data(df, scenario):
    # Run the physics simulator for each row.
    results = df.apply(lambda row: scenario.run_sim(row), axis=1, result_type='expand')

    labeled_df = df.copy()
    labeled_df['latency_base'] = results[0]
    labeled_df['latency_fused'] = results[1]

    # Calculate profit ratio for the AI target.
    labeled_df['profit_ratio'] = labeled_df['latency_base'] / labeled_df['latency_fused']
    labeled_df['hw_id'] = scenario.name

    return labeled_df

# Generate labeled datasets for both scenarios.
df_mem = label_data(raw_df, mem_scenario)
df_comp = label_data(raw_df, compute_scenario)

# Save the results to CSV files locally.
df_mem.to_csv("data/dataset_memory_bound.csv", index=False)
df_comp.to_csv("data/dataset_compute_bound.csv", index=False)

# Sync the generated data back to Google Drive.
!mkdir -p "{DRIVE_FOLDER}/data"
!cp -r data/* "{DRIVE_FOLDER}/data/"

print(f"Saved and synced records for {mem_scenario.name} and {compute_scenario.name}.")

In [ ]:
# @title Scenario distribution analysis

# Setup the visualization to compare scenarios.
plt.figure(figsize=(12, 6))

# Plot the memory-bound distribution in blue.
plt.hist(df_mem['profit_ratio'], bins=50, alpha=0.5, label='Memory-Bound', color='steelblue')

# Plot the compute-bound distribution in orange.
plt.hist(df_comp['profit_ratio'], bins=50, alpha=0.5, label='Compute-Bound', color='darkorange')

# Mark the efficiency threshold at 1.0.
plt.axvline(1.0, color='red', linestyle='--', linewidth=2)

# Set the axis labels and title.
plt.title("Optimization profit across hardware scenarios")
plt.xlabel("Profit ratio")
plt.ylabel("Frequency")
plt.legend()
plt.grid(axis='y', alpha=0.3)

plt.show()

# Display core metrics for verification.
print(f"Memory-Bound Median: {df_mem['profit_ratio'].median():.2f}")
print(f"Compute-Bound Median: {df_comp['profit_ratio'].median():.2f}")